<a href="https://colab.research.google.com/github/Taaniya/exploring-gpt2-language-model/blob/main/Text_classification_with_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers datasets accelerate

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import re
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.utils import resample

In [71]:
train_df = pd.read_csv("./drive/MyDrive/Colab Notebooks/NLP_exploration/datasets/train.csv")
test_df = pd.read_csv("./drive/MyDrive/Colab Notebooks/NLP_exploration/datasets/test.csv")

In [72]:
train_df.head()

,userName,content,score,appid,label
0,Barbie Villanueva,App updated 3 hours ago. Can no longer view an...,3,com.facebook.katana,1
1,Adipden Yonzon,my app stopped working shows a pending balance...,4,com.ubercab,1
2,A Google user,its a good app but the connection is pretty ba...,3,com.google.android.talk,1
3,Nicholas Jones,Love it. Still have all my text and number fro...,4,com.google.android.talk,1
4,siddique padinjareveetil,Good service and punctual,4,com.flipkart.android,1


In [73]:
train_df.count()

userName    28492
content     28491
score       28492
appid       28492
label       28492
dtype: int64

In [74]:
test_df.count()

userName    28494
content     28493
score       28494
appid       28494
label       28494
dtype: int64

In [75]:
train_df['label'].value_counts()

1    14246
0    14246
Name: label, dtype: int64

In [76]:
train_v2_df = resample(train_df, replace=False, n_samples=10000, random_state=5)
test_v2_df = resample(test_df, replace=False, n_samples=200, random_state=5)

In [77]:
train_v2_df['label'].value_counts()

1    5032
0    4968
Name: label, dtype: int64

In [78]:
test_v2_df['label'].value_counts()

1    103
0     97
Name: label, dtype: int64

In [79]:
updated_train_df = train_v2_df.replace({'label': {1: "positive", 0:" negative"}})

updated_train_df.head()

,userName,content,score,appid,label
9552,Soma Saha,I m just giving 3 star because insta gives me ...,3,com.instagram.android,positive
7575,1223 Ab,Good but will be better if for video chat able...,4,com.whatsapp,positive
18182,Dividale,App interface is absolutely terrible. I also e...,2,com.instagram.android,negative
9068,Alex S.,Instagram is my favorite social media platform...,4,com.instagram.android,positive
26581,Claudia 't Zand,Please give me the old version back. Ever sinc...,1,com.whatsapp,negative


In [80]:
updated_test_df = test_v2_df.replace({'label': {1: "positive", 0:" negative"}})

updated_test_df.head()

,userName,content,score,appid,label
9554,TIME PASS CORNER,Excellent app i can go anywhere at anytime whe...,4,com.olacabs.customer,positive
7580,Samim Ahmed,its nice and easy for hassle free travel,4,com.olacabs.customer,positive
16021,Himanshu Chandel,My whatsapp account not taking backup. And not...,2,com.whatsapp,negative
9070,Katherine Kat,It's a great app but lately I've been facing s...,3,com.instagram.android,positive
23725,Ashfak Malak,"Call quality is ok but video call is so bad , ...",2,com.whatsapp,negative


In [81]:
updated_train_df.drop(['userName', 'score', 'appid'], axis=1, inplace=True)
updated_test_df.drop(['userName', 'score', 'appid'], axis=1, inplace=True)

In [82]:
updated_train_df.rename(columns = {'content':'text'}, inplace = True)
updated_test_df.rename(columns = {'content':'text'}, inplace = True)

In [96]:
updated_train_df['short_text'] = updated_train_df['text'].str.slice(0,80)
updated_test_df['short_text'] = updated_test_df['text'].str.slice(0,80)

In [97]:
updated_train_df['short_text'].iloc[0]

'I m just giving 3 star because insta gives me so much of problem like i m postin'

In [98]:
updated_train_df.drop(['text' ], axis=1, inplace=True)
updated_test_df.drop(['text'], axis=1, inplace=True)

In [99]:
updated_train_df.rename(columns = {'short_text':'text'}, inplace = True)
updated_test_df.rename(columns = {'short_text':'text'}, inplace = True)

In [101]:
updated_train_df.head()

,label,text
9552,positive,I m just giving 3 star because insta gives me ...
7575,positive,Good but will be better if for video chat able...
18182,negative,App interface is absolutely terrible. I also e...
9068,positive,Instagram is my favorite social media platform...
26581,negative,Please give me the old version back. Ever sinc...


In [100]:
updated_test_df.head()

,label,text
9554,positive,Excellent app i can go anywhere at anytime whe...
7580,positive,its nice and easy for hassle free travel
16021,negative,My whatsapp account not taking backup. And not...
9070,positive,It's a great app but lately I've been facing s...
23725,negative,"Call quality is ok but video call is so bad , ..."


In [102]:
updated_train_df.count(), updated_test_df.count()

(label    10000
 text      9999
 dtype: int64,
 label    200
 text     200
 dtype: int64)

In [103]:
updated_train_df[updated_train_df.isnull().any(axis=1)]

,label,text
403,positive,NaN


In [104]:
updated_train_df.dropna(inplace=True)

In [105]:
updated_train_df.count()

label    9999
text     9999
dtype: int64

In [106]:
updated_train_df.head()

,label,text
9552,positive,I m just giving 3 star because insta gives me ...
7575,positive,Good but will be better if for video chat able...
18182,negative,App interface is absolutely terrible. I also e...
9068,positive,Instagram is my favorite social media platform...
26581,negative,Please give me the old version back. Ever sinc...


In [107]:
updated_train_df.to_csv("./drive/MyDrive/Colab Notebooks/NLP_exploration/datasets/train_v2.csv", index=False)
updated_test_df.to_csv("./drive/MyDrive/Colab Notebooks/NLP_exploration/datasets/test_v2.csv", index=False)

In [108]:
model_save_path = "./drive/MyDrive/Colab Notebooks/NLP_exploration/model_dir/distilgpt2-1"

train_dataset_path = "./drive/MyDrive/Colab Notebooks/NLP_exploration/datasets/train_v2.csv"
test_dataset_path = "./drive/MyDrive/Colab Notebooks/NLP_exploration/datasets/test_v2.csv"

In [109]:
!python drive/MyDrive/Colab\ Notebooks/NLP_exploration/finetune_gpt2_clm_pytorch.py \
--dataset-path "./drive/MyDrive/Colab Notebooks/NLP_exploration/datasets/train_v2.csv" \
--batch-size 16 \
--epoch 4 \
--model-name distilgpt2 \
--model-version 1 \
--model-dir "./drive/MyDrive/Colab Notebooks/NLP_exploration/model_dir"

2023-06-04 13:15:36.679617: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Extracting data files: 100% 1/1 [00:00<00:00, 124.59it/s]
Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4d3ae6cc2479f823/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.
100% 1/1 [00:00<00:00, 771.44it/s]
dataset loaded: DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 9999
    })
}) from path:./drive/MyDrive/Colab Notebooks/NLP_exploration/datasets/train_v2.csv
preprocessed dataset: ['###Text: I m just giving 3 star because insta gives me so much of problem like i m postin ###Label:positive', '###Text: Good but will be better if for video chat able to add more than 4 people ###Label:positive', '###Text: App interface is absolutely terrible. I also encounter a lot of bugs when return ###Label:negative', '###Text: Instagram i

In [110]:
def parse_label_preds(text):
  """ Parses generated text by the model to extracts the label.
      Regex pattern is defined based on the prompt patterns used in data preparation
      for model finetuning.
  """
  try:
    span = re.search("(###Text:)[a-zA-Z]+", text).span()
    label_text = text[span[0]: span[1]]
    label_text = re.sub("###Text:", "", label_text)

  except Exception as e:
    print(f"Can't parse: {text}")
    label_text = "None"

  return label_text

In [111]:
def prep_test_data(row):
  """ Preprocessing test data example the same way during inference as is done 
  during data prep for finetuning.
  """
  row['text'] = "###Text: " + row['text'] + " ###Label:"
  return row

In [112]:
# Load model with text generation NLP task for inference on GPU
ft_gpt2_classifier = pipeline("text-generation", model=model_save_path, device=0)

# load test datasets
test_data = load_dataset("csv", data_files={"test": [test_dataset_path]}, 
                         download_mode='force_redownload')

test_df = pd.read_csv(test_dataset_path)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-392fcd31aaedc039/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [113]:
updated_test_dataset = test_data.map(prep_test_data)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [114]:
updated_test_dataset['test']['text'][:10]

['###Text: Excellent app i can go anywhere at anytime where i want to go i am very pride of ###Label:',
 '###Text: its nice and easy for hassle free travel ###Label:',
 '###Text: My whatsapp account not taking backup. And not showing my email account also. ###Label:',
 "###Text: It's a great app but lately I've been facing some glitches on my posts like i ca ###Label:",
 "###Text: Call quality is ok but video call is so bad , can't hear and see clearly ###Label:",
 "###Text: Mine never updates like my husband's does and daughters does it stays the same a ###Label:",
 '###Text: Too invasive. The constant notifications and the lack of a clear way to turn the ###Label:',
 '###Text: I had this app for ages to talk to my family and all that but out of nowhere, ev ###Label:',
 '###Text: 25 years experience in India & Middle East in Project Management & Coordination; ###Label:',
 '###Text: Order history is empty now! Where are my orders Flipkart? ###Label:']

In [116]:
prompt = "###Text: The app is good. I like it ###Label:"
output = ft_gpt2_classifier(prompt, return_full_text=True, no_repeat_ngram_size=2, 
                            num_beams=2)
print(output[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


###Text: The app is good. I like it ###Label: __________________________________________________________________________________________________________________ __________________________________________ I'm using ____________ ____________________ I've ____________________________ i'm a _____  _______ ______ ___ 


In [ ]:
preds = []
# Run inference on entire test dataset. Run on each example separately rather 
# than in batches for GPT2 with text generation task while using KeyDataset

for output in ft_gpt2_classifier(KeyDataset(updated_test_dataset['test'], 'text'), 
                                 return_full_text=True, 
                                 no_repeat_ngram_size=2, 
                                 num_beams=2):
  pred_label = parse_label_preds(output[0]['generated_text'])
  preds.append(pred_label)

In [119]:
test_df['preds'] = preds

In [120]:
print(test_df[["label", "preds"]].head(20))

        label preds
0    positive  None
1    positive  None
2    negative  None
3    positive  None
4    negative  None
5    negative  None
6    negative  None
7    positive  None
8    positive  None
9    negative  None
10   positive  None
11   negative  None
12   negative  None
13   positive  None
14   negative  None
15   positive  None
16   negative  None
17   negative  None
18   negative  None
19   positive  None


In [121]:
print(classification_report(test_df['label'], test_df['preds']))

              precision    recall  f1-score   support

    negative       0.00      0.00      0.00      97.0
        None       0.00      0.00      0.00       0.0
    positive       0.00      0.00      0.00     103.0

    accuracy                           0.00     200.0
   macro avg       0.00      0.00      0.00     200.0
weighted avg       0.00      0.00      0.00     200.0



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [ ]:
# Check misclassified examples
test_df[test_df['preds'] != test_df['label']]

In [124]:
test_df['preds'].value_counts()

None    200
Name: preds, dtype: int64

In [69]:
# Save misclassified cases into excel for further RCA
test_df[test_df['preds'] != test_df['label']].to_excel("./misclassified.xlsx", index=False)